In [13]:
import pandas as pd
import numpy as np
df = pd.read_csv('/Users/pranjal/Desktop/Causal-Inference/data/nyvoucher.csv')

In [14]:
df.head()

,s_id,voucher,pre_ach,post_ach
0,42,0,74.0,83.0
1,194,0,7.5,4.0
2,218,1,2.5,3.5
3,261,1,0.0,26.5
4,304,1,11.0,2.0


In [15]:
df = pd.read_csv('/Users/pranjal/Desktop/Causal-Inference/data/nyvoucher.csv')

In [16]:
#import wooldridge
#df = wooldridge.data('jtrain3')
#df['avg'] = 0.5 * (df.re74+df.re75)
#df = df.dropna()
df = df.fillna(0)
#df = df[df.avg <= 15]
y = df[outcome]
d = df[treatment]
x = df[rest].astype('float')
print(df.shape)
df.head()

(521, 3)


,post_ach,voucher,s_id
0,83.0,0,42
1,4.0,0,194
2,3.5,1,218
3,26.5,1,261
4,2.0,1,304


In [17]:
# Simple Comparision of Means
import statsmodels.api as sm
mod = sm.OLS(y, sm.add_constant(np.c_[d], prepend=False))
res = mod.fit()
print(res.summary())
print(res.params[0])
print(res.bse[0])

                            OLS Regression Results                            
Dep. Variable:               post_ach   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                  0.014
Method:                 Least Squares   F-statistic:                     8.475
Date:                Tue, 06 Dec 2022   Prob (F-statistic):            0.00375
Time:                        13:51:45   Log-Likelihood:                -2274.3
No. Observations:                 521   AIC:                             4553.
Df Residuals:                     519   BIC:                             4561.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             4.8988      1.683      2.911      0.0

In [18]:
# Pooled Regression Adjustment
import statsmodels.api as sm
mod = sm.OLS(y, sm.add_constant(np.c_[d, x], prepend=False))
res = mod.fit()
print(res.summary())
print(res.params[0])
print(res.bse[0])

                            OLS Regression Results                            
Dep. Variable:               post_ach   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                  0.012
Method:                 Least Squares   F-statistic:                     4.278
Date:                Tue, 06 Dec 2022   Prob (F-statistic):             0.0144
Time:                        13:51:46   Log-Likelihood:                -2274.2
No. Observations:                 521   AIC:                             4554.
Df Residuals:                     518   BIC:                             4567.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             4.9337      1.688      2.923      0.0

In [42]:
# DML regression - still yeilds unbiased estimate of ATE 
from econml.dml import LinearDML
est = LinearDML(random_state=1)
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
est = LinearDML(discrete_treatment=False, cv = 5)
est.fit(y.ravel(), d.ravel(), X=None,W=x)
est.summary()

Coefficient Results:  X is None, please call intercept_inference to learn the constant!


,point_estimate,stderr,zstat,pvalue,ci_lower,ci_upper
cate_intercept,4.768,1.663,2.867,0.004,1.509,8.028


In [47]:
# DML regression - still yeilds unbiased estimate of ATE 
from econml.dml import LinearDML
est = LinearDML(random_state=1)
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
est = LinearDML(discrete_treatment=False, model_y = RandomForestRegressor(), model_t = RandomForestRegressor(), cv = 5)
est.fit(y.ravel(), d.ravel(), X=None,W=x)
est.summary()

Coefficient Results:  X is None, please call intercept_inference to learn the constant!


,point_estimate,stderr,zstat,pvalue,ci_lower,ci_upper
cate_intercept,3.402,1.866,1.823,0.068,-0.256,7.06


In [12]:
import numpy as np
from doubleml.datasets import make_plr_CCDDHNR2018
from doubleml import DoubleMLData

np.random.seed(1234)
dml_data_bonus = DoubleMLData(df, y_col=outcome,
                                  d_cols=treatment,
                                  x_cols=list(rest))
print(dml_data_bonus)
from sklearn.base import clone
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LassoCV
learner = RandomForestRegressor(n_estimators = 500, max_features = 'sqrt', max_depth= 6)
ml_l_bonus = clone(learner)
ml_m_bonus = clone(learner)
learner = LassoCV()
ml_l_sim = clone(learner)
ml_m_sim = clone(learner)
def non_orth_score(y, d, l_hat, m_hat, g_hat, smpls):
    u_hat = y - g_hat
    psi_a = -np.multiply(d, d)
    psi_b = np.multiply(d, u_hat)
    return psi_a, psi_b

from doubleml import DoubleMLPLR
np.random.seed(3141)
obj_dml_plr_bonus = DoubleMLPLR(dml_data_bonus, ml_l_bonus, ml_m_bonus)
obj_dml_plr_bonus.fit();
print(obj_dml_plr_bonus)

================== DoubleMLData Object ==================

------------------ Data summary      ------------------
Outcome variable: post_ach
Treatment variable(s): ['voucher']
Covariates: ['s_id']
Instrument variable(s): None
No. Observations: 521

------------------ DataFrame info    ------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 521 entries, 0 to 520
Columns: 3 entries, post_ach to s_id
dtypes: float64(1), int64(2)
memory usage: 12.3 KB

================== DoubleMLPLR Object ==================

------------------ Data summary      ------------------
Outcome variable: post_ach
Treatment variable(s): ['voucher']
Covariates: ['s_id']
Instrument variable(s): None
No. Observations: 521

------------------ Score & algorithm ------------------
Score function: partialling out
DML algorithm: dml2

------------------ Machine learner   ------------------
Learner ml_l: RandomForestRegressor(max_depth=6, max_features='sqrt', n_estimators=500)
Learner ml_m: RandomForestRegre

In [21]:
import numpy as np
from doubleml.datasets import make_plr_CCDDHNR2018
from doubleml import DoubleMLData

np.random.seed(1234)
dml_data_bonus = DoubleMLData(df, y_col=outcome,
                                  d_cols=treatment,
                                  x_cols=list(rest))
print(dml_data_bonus)
from sklearn.base import clone
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LassoCV
learner = RandomForestRegressor(n_estimators = 100, max_features = 'sqrt', max_depth= 6)
ml_l_bonus = clone(learner)
ml_m_bonus = clone(learner)
learner = LassoCV()
ml_l_sim = clone(learner)
ml_m_sim = clone(learner)
def non_orth_score(y, d, l_hat, m_hat, g_hat, smpls):
    u_hat = y - g_hat
    psi_a = -np.multiply(d, d)
    psi_b = np.multiply(d, u_hat)
    return psi_a, psi_b

from doubleml import DoubleMLPLR
np.random.seed(3141)
obj_dml_plr_bonus = DoubleMLPLR(dml_data_bonus, ml_l_bonus, ml_m_bonus)
obj_dml_plr_bonus.fit();
print(obj_dml_plr_bonus)

================== DoubleMLData Object ==================

------------------ Data summary      ------------------
Outcome variable: post_ach
Treatment variable(s): ['voucher']
Covariates: ['s_id']
Instrument variable(s): None
No. Observations: 521

------------------ DataFrame info    ------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 521 entries, 0 to 520
Columns: 3 entries, post_ach to s_id
dtypes: float64(1), int64(2)
memory usage: 12.3 KB

================== DoubleMLPLR Object ==================

------------------ Data summary      ------------------
Outcome variable: post_ach
Treatment variable(s): ['voucher']
Covariates: ['s_id']
Instrument variable(s): None
No. Observations: 521

------------------ Score & algorithm ------------------
Score function: partialling out
DML algorithm: dml2

------------------ Machine learner   ------------------
Learner ml_l: RandomForestRegressor(max_depth=6, max_features='sqrt')
Learner ml_m: RandomForestRegressor(max_depth=6, 

In [20]:
# DML regression - still yeilds unbiased estimate of ATE 
from econml.dml import LinearDML
est = LinearDML(random_state=45)
est.fit(y, d, X=None,W=x)
est.summary()

Coefficient Results:  X is None, please call intercept_inference to learn the constant!


,point_estimate,stderr,zstat,pvalue,ci_lower,ci_upper
cate_intercept,4.857,1.644,2.954,0.003,1.634,8.079
